<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# 1.  [Apprentissage en Grande Dimension](https://github.com/wikistat/High-Dimensional-Learning ):  [Reconnaissance d'Activité Humaine](https://github.com/wikistat/High-Dimensional-Statistics/tree/master/HumanActivityRecognition) ([*HAR*](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)) en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>    Detection d'anomalies

## 1.1 Contexte
Les données sont issues de la communauté qui vise la reconnaissance d'activités humaines (*Human activity recognition, HAR*) à partir d’enregistrements, par exemple du gyroscope et de l'accéléromètre d'un smartphone, objet connecté précurseur et dont la fonctionnalité de téléphonie devient très secondaire.
Voir à ce propos l'[article](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-11.pdf) relatant un colloque de 2013.  

Les données publiques disponibles et largement étudiées ont été acquises, décrites et analysées par [Anguita et al. (2013)]().
Elles sont accessibles sur le [dépôt](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) de l'University California Irvine (UCI) consacré à l'apprentissage machine ainsi que sur le site *Kaggle*.

L'archive contient les données brutes: accélérations en x, y, et z, chacun de 128 colonnes. D'autres fichiers en y soustrayant la gravité naturelle ainsi que les accélérations angulaires en x, y, et z soit en tout 9 fichiers. Mais 6 utiles avec 6*128=768 mesures.


Ce notebook présente la partie détection d'anomalies sur les données métiers et sur les brutes. 

## 1.2 Objectifs : 

-Appliquer différentes méthodes de détection d'anomalies sur des données vectorielles (données métiers): 

  - Classification ascendante hiérarchique
  - One-class SVM
  - Local Outlier Factor
  - Isolation Forest
 
-Détection d'anomalies sur des données fonctionnelles : transformation des données fonctionnelles afin de définir des caractéristiques (ACP, transformation en ondelettes, FFT) sur lesquelles on applique les méthodes de détection d'anomalies 

-Comparaison des différentes méthodes  sur les données HAR. 

# 2. Introduction

## 2.1 Téléchargement des librairies

In [ ]:
import pandas as pd
import numpy as np

# ACP
import sklearn.decomposition as sd
import sklearn.preprocessing as sp

# Hierarchical clustering
import scipy.cluster.hierarchy as sch
# LOF
import sklearn.neighbors as sn
# Isolation Forest
import sklearn.ensemble as se

# Plot et Display
import utils.illustration as uil
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()
sb.set_style("whitegrid")

In [ ]:
SIGNALS = [ "body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z"]
CMAP = plt.get_cmap("Set1")
ACTIVITY_DIC = {1 : "WALKING",
2 : "WALKING UPSTAIRS",
3 : "WALKING DOWNSTAIRS",
4 : "SITTING",
5 : "STANDING",
6 : "LAYING"}
COLOR_DIC = {v:CMAP(k-2) if v!="WALKING" else CMAP(10) for k,v in ACTIVITY_DIC.items()}

## 2.2 Les données
### 2.2.1 Téléchargement des données


In [ ]:
import utils.load as ul

#Multidimensional Data
X_train = ul.load_signals("train", SIGNALS)
Y_train_label = ul.load_y("train")
X_train_metier= ul.my_read_csv("train/X_train.txt").values

### 2.2.2 Constitution des jeux de données avec anomalies

On construit un jeu de données constitué de 

   * `N_normal` signaux considérés comme normaux (associés au comportement *WALKING*).  
   * `N_anormal` signaux par type de signaux anormaux (*WALKING UPSTAIRS*, WALKING DOWNSTAIRS*, *SITTING*, *STANDING*, *LAYING*)


In [ ]:

N_normal = 800
N_anormal = 2

# New Y Label
Y= np.hstack([np.repeat(1,N_normal)] + [np.repeat(i, N_anormal) for i in range(2,7)])
Y_label = np.array([ACTIVITY_DIC[y] for y in Y])
#New X Data
index_per_act = np.hstack([np.where(Y_train_label==1)[0][:N_normal]] + [np.where(Y_train_label==act)[0][:N_anormal] for act in range(2,7)])

X = X_train[index_per_act]
X_metier = X_train_metier[index_per_act]


Pour chaque type de signal, on affiche un echantillon des comportement normaux, ainsi que les différentes anomalies.

In [ ]:
nb_sample_per_activity = dict([(v,50) if v=="WALKING" else (v,N_anormal) for k,v in ACTIVITY_DIC.items()])
linestyle_per_activity = dict([(v,"dashed") if v=="WALKING" else (v,"solid") for k,v in ACTIVITY_DIC.items()])
linewidth_per_activity = dict([(v,1) if v=="WALKING" else (v,2) for k,v in ACTIVITY_DIC.items()])

fig = plt.figure(figsize=(16,18))    
uil.plot_signaux(fig, X, Y_label, SIGNALS, COLOR_DIC, nb_sample_per_activity, 
             linestyle_per_activity, linewidth_per_activity, figdim1 = 3, figdim2 = 2, legend=True)


## 2.3 Analyse en composantes principales
### 2.3.1  Sur un signal : l'accélération en x

In [ ]:
isignal = 0
print("ACP on signal " + SIGNALS[isignal])
X_signal = np.vstack([x[:,isignal] for x in X])

acp = sd.PCA()
X_acp_signal = acp.fit_transform(sp.scale(X_signal))

X_signal.shape

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, acp, X_acp_signal)

In [ ]:
N = X.shape[0]
colors=[COLOR_DIC[y] for y in Y_label]
markersizes = [60 if y==1 else 140 for y in Y]
fig = plt.figure(figsize=(15,10))
uil.plot_projection_acp(fig, X_acp_signal, acp, colors=colors, markersizes = markersizes, color_dic=COLOR_DIC)

*Q* Commenter les résultats dans la perpective de la détection d'anomalies

### 2.3.2  Sur tous les signaux

In [ ]:
X_signaux = np.vstack([x.reshape(128*6) for x in X])
acp = sd.PCA()
X_acp_signaux = acp.fit_transform(sp.scale(X_signaux))

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, acp, X_acp_signaux)

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_projection_acp(fig, X_acp_signaux, acp, colors, markersizes, color_dic=COLOR_DIC)

### 2.3.3 Sur les données métiers

In [ ]:
acp = sd.PCA()
X_acp_metier = acp.fit_transform(sp.scale(X_metier))

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, acp, X_acp_metier)

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_projection_acp(fig, X_acp_metier, acp, colors, markersizes, color_dic=COLOR_DIC)

*Q* Commenter les résultats dans la perpective de la détection d'anomalies

# 3. Détection d'anomalies sur les données "métiers"

Il semble assez aisé de détecter les anomalies sur les données "métiers". Nous appliquons les méthodes classique : Classification ascendante hiérarchqie avec l'option "single", One class SVM, Local Outlier Factor et Isolation Forest. 
Les différentes méthodes n'ont pas nécessairement été optimisées. Etudiez  l'impact des différents paramètres sur la détection d'anomalies. 

## 3.1 Classification Ascendante Hiérarchique

In [ ]:
Z = sch.linkage(X_metier, 'single')
C = np.array([c[0] for c in sch.cut_tree(Z,6)])

CT_HCA = pd.DataFrame(list(zip(C,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_HCA.pred, CT_HCA.Anomaly))

In [ ]:
LABELS = ["" if y=="WALKING" else y for y in Y_label]
fig = plt.figure(figsize=(25, 10))
sch.dendrogram( Z, p=6, leaf_rotation=45.,leaf_font_size=15,labels=LABELS, truncate_mode="level"  # font size for the x axis labels
)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')

ax =fig.get_axes()[0]
xlbls = ax.get_xmajorticklabels()
for lbl in xlbls:
    if lbl.get_text() in COLOR_DIC:
        lbl.set_color(COLOR_DIC[lbl.get_text()])


plt.show()

## 3.2  One class SVM

In [ ]:
import sklearn.svm as ssvm
OCS = ssvm.OneClassSVM(kernel="rbf", nu=0.05)

OCS.fit(X_metier)
pred = OCS.predict(X_metier)

CT_svm = pd.DataFrame(list(zip(pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_svm.pred, CT_svm.Anomaly))


In [ ]:
fig = plt.figure(figsize=(30,10))
ax = fig.add_subplot(1,1,1)
uil.plot_detection_result(fig, ax, CT_svm,COLOR_DIC, normal_behaviour="WALKING")


## 3.3 Local Outlier Factor

In [ ]:
contamination=0.05
metric = "euclidean"
n_neighbors = 15
clf = sn.LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, metric=metric)
y_pred = clf.fit_predict(X_metier)

CT_metier_lof = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_metier_lof.pred, CT_metier_lof.Anomaly))


In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)
uil.plot_detection_result(fig, ax, CT_metier_lof, COLOR_DIC, normal_behaviour="WALKING")


## 3.4 Isolation Forest

In [ ]:
clf = se.IsolationForest(n_estimators=100, contamination=0.05, bootstrap=True, n_jobs=-1)

clf.fit(X_metier)
y_pred = clf.predict(X_metier)

CT_IF = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_IF.pred, CT_IF.Anomaly))

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)
uil.plot_detection_result(fig, ax, CT_IF, COLOR_DIC, normal_behaviour="WALKING")


**Q. ** Quelle est votre conclusion sur les données métier ? 

L'objectif de la section suivante est d'essayer de détecter les anomalies sur les données  brutes.

# 4. Détection d'anomalies sur les signaux 

## 4.1 Classification Ascendante Hiérarchique

On travaille tout d'abord sur un signal : l'accélération en x. 


In [ ]:
Z = sch.linkage(X_signal,'single')

C = np.array([c[0] for c in sch.cut_tree(Z,6)])

CT_HCA = pd.DataFrame(list(zip(C,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_HCA.pred, CT_HCA.Anomaly))

In [ ]:
LABELS = ["" if y=="WALKING" else y for y in Y_label]
fig = plt.figure(figsize=(25, 10))
sch.dendrogram( Z, p=6, leaf_rotation=45.,leaf_font_size=15,labels=LABELS, truncate_mode="level"  # font size for the x axis labels
)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')

ax =fig.get_axes()[0]
xlbls = ax.get_xmajorticklabels()
for lbl in xlbls:
    if lbl.get_text() in COLOR_DIC:
        lbl.set_color(COLOR_DIC[lbl.get_text()])


plt.show()

**Q.** Combien d anomalies a-t-on détecté ? Obtient-on de meilleurs résultats en prenant tous les signaux ?

## 4.2 One class SVM

### 4.2.1 Sur les deux premières composantes de l'ACP

In [ ]:
import sklearn.svm as ssvm
OCS = ssvm.OneClassSVM(kernel="rbf", nu=0.01)

OCS.fit(X_acp_signal[:,:2])
pred = OCS.predict(X_acp_signal[:,:2])

CT_svm = pd.DataFrame(list(zip(pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_svm.pred, CT_svm.Anomaly))

**Q.** Commentez les résultats. Obtient-on de meilleurs résultats en augmentant le nombre de composantes ? 

In [ ]:
X_acp = X_acp_signal
nu = 0.02

# fit the model
clf = ssvm.OneClassSVM(kernel="rbf",nu=nu)
clf.fit(X_acp[:,:2])
y_pred_train = clf.predict(X_acp[:,:2])


fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)

markersizes = [10 if y==1 else 20 for y in Y]
labels = [""] * N 
for il, l in [(np.where(Y_label==y)[0][0],y) for y in set(Y_label)]:
    labels[il] = l

uil.plot_decision_function(fig, ax, clf, X_acp, y_pred_train, colors=colors, labels = labels, markersizes=markersizes)
ax.set_title("Novelty Detection : nu=%.1f" %nu)


In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)
uil.plot_detection_result(fig, ax, CT_svm, COLOR_DIC, normal_behaviour="WALKING")


### 4.2.2 Sur l'accélération en x :

In [ ]:
import sklearn.svm as ssvm
OCS = ssvm.OneClassSVM(kernel="rbf", nu=0.05)

OCS.fit(X_signal)
pred = OCS.predict(X_signal)

CT_svm = pd.DataFrame(list(zip(pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_svm.pred, CT_svm.Anomaly))

**Q.** Commenter les résultats. L'application de la méthode sur tous les signaux améliore-t-elle les résultats ? 

## 4.3 Local Outlier Factor

### 4.3.1 Sur l'accélération en x : 

In [ ]:
contamination=0.05
metric = "euclidean"
n_neighbors = 15
clf = sn.LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, metric=metric)
y_pred = clf.fit_predict(X_signal)

CT_lof = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_lof.pred, CT_lof.Anomaly))


In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)
uil.plot_detection_result(fig, ax, CT_lof, COLOR_DIC, normal_behaviour="WALKING")


In [ ]:
contamination=0.05
metric = "euclidean"
n_neighbors = 15
clf = sn.LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, metric=metric)
y_pred = clf.fit_predict(X_signaux)

CT_tous_lof = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_tous_lof.pred, CT_tous_lof.Anomaly))


**Q.** Commenter les résultats. Obtient-on de meilleures performances sur l'ensemble des signaux ? En modifiant les paramètres ? 

### 4.3.2 Sur les composantes de l'ACP : 

In [ ]:
X_acp = X_acp_signal
n_neighbors = 15

# fit the model
clf = sn.LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, metric = metric)
y_pred = clf.fit_predict(X_acp[:,:2])

CT_ACP_lof = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_lof.pred, CT_ACP_lof.Anomaly))

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)
uil.plot_decision_function(fig, ax, clf, X_acp, y_pred, method_name="LOF", colors=colors, labels = labels, markersizes=markersizes)
ax.set_title("Number of neighbors : %d" %n_neighbors)

## 4.4 Isolation Forest

### 4.4.1 Sur l'accélération en x :

In [ ]:
clf = se.IsolationForest(n_estimators=100, contamination=0.05, bootstrap=True, n_jobs=-1)

#clf.fit(X_acp_metier)
#y_pred = clf.predict(X_acp_metier)

clf.fit(X_signal)
y_pred = clf.predict(X_signaux)

CT_IF = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_IF.pred, CT_IF.Anomaly))

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)
uil.plot_detection_result(fig, ax, CT_IF, COLOR_DIC, normal_behaviour="WALKING")


### 4.4.2 Sur les composantes de l'ACP 

In [ ]:
X_acp = X_acp_signal

contamination=0.05
clf = se.IsolationForest(n_estimators=100, contamination=contamination, bootstrap=True, n_jobs=-1)
clf.fit(X_acp[:,:2])
y_pred = clf.predict(X_acp[:,:2])

CT_IF = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_IF.pred, CT_IF.Anomaly))



**Q.** Obtient-on de meilleurs résultats en augmentant le nombre de composantes de l'ACP ? 

En conclusion, les méthodes de détection d'anomalies appliquées directement sur les signaux ne fonctionnent pas bien. Nous allons voir si la projection sur une base d'ondelettes permet d'obtenir de meilleurs résultats. 

# 5. Détection d'anomalies sur la décomposition en ondelettes  


## 5.1 Décomposition en ondelettes

On travaille sur l'accélération en x. 

In [ ]:
import pywt
from pywt import wavedec

from statsmodels.robust import mad

In [ ]:
isignal = 0
print(" signal " + SIGNALS[isignal])
X_signal = np.vstack([x[:,isignal] for x in X])



In [ ]:

wf = "haar"

Coeff = []
TCoeff = []
for x in X_signal :
    #Apply wavelet decomposition
    coeffs = pywt.wavedec(x,wf,level=7)
    coeffs_flatten = np.hstack(coeffs)
    Coeff.append(coeffs_flatten)
    
    # Compute universal Threshold http://jseabold.net/blog/2012/02/23/wavelet-regression-in-python/
    sigma = mad(coeffs[-1])
    uthresh = sigma*np.sqrt(2*np.log(128))
    # Apply Threshold on 4 last levels
    coeffs_thresh = [pywt.threshold(c, uthresh, mode="hard") if i<=3 else c for i,c in enumerate(coeffs[::-1])]
    coeffs_thresh_flatten = np.hstack(coeffs_thresh[::-1])
    TCoeff.append(coeffs_thresh_flatten)
    
Coeff = np.array(Coeff)
TCoeff = np.array(TCoeff)
print(Coeff.shape, TCoeff.shape)
print(np.sum(Coeff!=0), np.sum(TCoeff!=0))

On conserve seulement les coefficients de niveau 1 à 4, les autres sont considérés comme du bruit et annulés. 

In [ ]:
#Coefficient de niveau 1 à 4 : 
CoeffA4=Coeff[:,:16]

## 5.2 ACP des coefficients d'ondelettes

In [ ]:
acp = sd.PCA()
X_acp_ond = acp.fit_transform(sp.scale(Coeff))


In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, acp, X_acp_ond)

In [ ]:
markersizes = [60 if y==1 else 140 for y in Y]
fig = plt.figure(figsize=(15,10))
uil.plot_projection_acp(fig, X_acp_ond, acp, colors, markersizes, color_dic=COLOR_DIC)

## 5.3 ACP des coefficients d'ondelettes de niveau 1 à 4 :  

In [ ]:
acp = sd.PCA()
X_acp_ondA4 = acp.fit_transform(sp.scale(CoeffA4))

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, acp, X_acp_ondA4)

In [ ]:
markersizes = [60 if y==1 else 140 for y in Y]
fig = plt.figure(figsize=(15,10))
uil.plot_projection_acp(fig, X_acp_ondA4, acp, colors, markersizes, color_dic=COLOR_DIC)

*Q* Commenter ces résultats en vue de la détection d'anomalies

## 5.4 Classification ascendante hiérarchique

In [ ]:
Z = sch.linkage(Coeff, 'single')
C = np.array([c[0] for c in sch.cut_tree(Z,6)])

CT_HCA = pd.DataFrame(list(zip(C,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_HCA.pred, CT_HCA.Anomaly))

In [ ]:
LABELS = ["" if y=="WALKING" else y for y in Y_label]
fig = plt.figure(figsize=(25, 10))
sch.dendrogram( Z, p=6, leaf_rotation=45.,leaf_font_size=15,labels=LABELS, truncate_mode="level"  # font size for the x axis labels
)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')

ax =fig.get_axes()[0]
xlbls = ax.get_xmajorticklabels()
for lbl in xlbls:
    if lbl.get_text() in COLOR_DIC:
        lbl.set_color(COLOR_DIC[lbl.get_text()])


plt.show()

## 5.5 One class SVM

### 5.5.1 Sur tous les coefficients 

In [ ]:
import sklearn.svm as ssvm
OCS = ssvm.OneClassSVM(kernel="rbf", nu=0.05)

OCS.fit(Coeff)
pred = OCS.predict(Coeff)

CT_svm = pd.DataFrame(list(zip(pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_svm.pred, CT_svm.Anomaly))

**Q.** Obtient-on de meilleurs résultats avec les coefficients seuillés ? Avec les coefficients de niveau 1 à 4 ? 

## 5.6 Local Outlier Factor

### 5.6.1 Sur  les coefficients de niveaux 1 à 4

In [ ]:
contamination=0.05
metric = "euclidean"
n_neighbors = 15
clf = sn.LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, metric=metric)
y_pred = clf.fit_predict(CoeffA4)

CT_ond_lof = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_ond_lof.pred, CT_ond_lof.Anomaly))


In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)
uil.plot_detection_result(fig, ax, CT_ond_lof, COLOR_DIC, normal_behaviour="WALKING")


**Q.** Obtient-on de meilleurs résultats avec les tous les coefficients ? avec les coefficients seuillés ? Avec les coefficients de niveau  plus élevé ? 

## 5.7 Isolation Forest

### 5.7.1 Sur tous les coefficients

In [ ]:
clf = se.IsolationForest(n_estimators=100, contamination=0.05, bootstrap=True, n_jobs=-1)

clf.fit(Coeff)
y_pred = clf.predict(Coeff)

CT_IF = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_IF.pred, CT_IF.Anomaly))

**Q.** Obtient-on de meilleurs résultats avec les coefficients seuillés ? Avec les coefficients de niveau 1 à 4 ? 

**Q.** Nous avons travaillé seulement sur l'accélération en x.  Considérer l'ensemble des signaux améliore-t-il les résultats ? 

**Conclusion :** Les méthodes de détection d'anomalies considérées appliquées sur  signaux bruts, comme les coefficinets d'ondelettes ne permettent pas de détecter les signaux atypiques. 
Nous allons utiliser la transformée de Fourier. 

# 6. Détection d'anomalies sur les coefficients de la FFT 


In [ ]:
# Coefficients fft : 

from scipy.fftpack import fft

isignal = 0
print(" signal " + SIGNALS[isignal])
X_signal = np.vstack([x[:,isignal] for x in X])

#print(amplitudefft)
#plt.plot(amplitudefft)

fftCoeff = []

for x in X_signal :
    
    mx=np.mean(x)
    x_centre=x-mx
   #Apply fast Fourier transform
    coeffsfft=np.abs(fft(x_centre))  
    coeffsfft_flatten = np.hstack(coeffsfft)
    fftCoeff.append(coeffsfft_flatten)
        
fftCoeff = np.array(fftCoeff)

# Il suffit de garder la moitié des coefficients (ils sont ensuite répétés  de manière symétrique)

fftCoeff=fftCoeff[:,:64]
print(fftCoeff.shape)
print(np.sum(fftCoeff!=0))

## 6.1 ACP des coefficients FFT

In [ ]:
acp = sd.PCA()
X_acp_fft = acp.fit_transform(sp.scale(fftCoeff))


In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, acp, X_acp_fft)

In [ ]:

fig = plt.figure(figsize=(15,10))
uil.plot_projection_acp(fig, X_acp_fft, acp, colors, markersizes, color_dic=COLOR_DIC)

*Q* Quels signaux se distinguent bien des autres ? Est-ce cohérent avec ce qui a été vu dans les calepins précédents ? 

## 6.2 Classification ascendante hiérarchique

In [ ]:
Z = sch.linkage(fftCoeff, 'single')
C = np.array([c[0] for c in sch.cut_tree(Z,6)])

CT_HCA = pd.DataFrame(list(zip(C,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_HCA.pred, CT_HCA.Anomaly))

In [ ]:
LABELS = ["" if y=="WALKING" else y for y in Y_label]
fig = plt.figure(figsize=(25, 10))
sch.dendrogram( Z, p=6, leaf_rotation=45.,leaf_font_size=15,labels=LABELS, truncate_mode="level"  # font size for the x axis labels
)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')

ax =fig.get_axes()[0]
xlbls = ax.get_xmajorticklabels()
for lbl in xlbls:
    if lbl.get_text() in COLOR_DIC:
        lbl.set_color(COLOR_DIC[lbl.get_text()])


plt.show()

**Q** Commentez les résultats. 

## 6.3 One class SVM

In [ ]:
import sklearn.svm as ssvm
OCS = ssvm.OneClassSVM(kernel="rbf", nu=0.05)

OCS.fit(fftCoeff)
pred = OCS.predict(fftCoeff)

CT_FFT_svm = pd.DataFrame(list(zip(pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_FFT_svm.pred, CT_FFT_svm.Anomaly))

**Q** Etudiez l'impact du noyau et du paramètre nu. 

## 6.4 Local Outlier Factor

In [ ]:
contamination=0.05
metric = "euclidean"
n_neighbors = 15
clf = sn.LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, metric=metric)
y_pred = clf.fit_predict(fftCoeff)

CT_FFT_lof = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_FFT_lof.pred, CT_FFT_lof.Anomaly))


In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)
uil.plot_detection_result(fig, ax, CT_FFT_lof, COLOR_DIC, normal_behaviour="WALKING")


** Q** Commentez les résultats et regardez l'impact des paramètre. 

## 6.5 Isolation Forest

In [ ]:
clf = se.IsolationForest(n_estimators=100, contamination=0.05, bootstrap=True, n_jobs=-1)

clf.fit(fftCoeff)
y_pred = clf.predict(fftCoeff)

CT_IF = pd.DataFrame(list(zip(y_pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_IF.pred, CT_IF.Anomaly))


## 6.6 Visualisation des résultats de LOF 

La méthode LOF est l'une des plus performantes quelque soit le type de "features" considérées. On visualise les résultats de cette méthode pour les différents cas considérés dans le calepin. 


In [ ]:
fig = plt.figure(figsize=(40,40))
ax = fig.add_subplot(3,2,1)
uil.plot_detection_result(fig, ax, CT_lof, COLOR_DIC, normal_behaviour="WALKING")
ax.set_title("Donnees Brutes- Boddy acc x ", fontsize=25)
ax = fig.add_subplot(3,2,2)
uil.plot_detection_result(fig, ax, CT_tous_lof, COLOR_DIC, normal_behaviour="WALKING")
ax.set_title("Donnees Brutes- tous les signaux", fontsize=25)
ax = fig.add_subplot(3,2,3)
uil.plot_detection_result(fig, ax, CT_ACP_lof, COLOR_DIC, normal_behaviour="WALKING")
ax.set_title("ACP, 2 premieres composantes", fontsize=25)
ax = fig.add_subplot(3,2,4)
uil.plot_detection_result(fig, ax, CT_ond_lof, COLOR_DIC, normal_behaviour="WALKING")
ax.set_title("Ondelettes, coefficients niveaux 1 à 4 ", fontsize=25)
ax = fig.add_subplot(3,2,5)
uil.plot_detection_result(fig, ax, CT_metier_lof, COLOR_DIC, normal_behaviour="WALKING")
ax.set_title("Données métiers", fontsize=25)
ax = fig.add_subplot(3,2,6)
uil.plot_detection_result(fig, ax, CT_FFT_lof, COLOR_DIC, normal_behaviour="WALKING")
ax.set_title("FFT", fontsize=25)

# 7. Conclusion

Nous avons étudié diverses méthodes de détection d'anomalies. Sur les données "métiers", il est assez simple de détecter les données atypiques. Sur les données fonctionnelles, nous avons vu l'importance de définir de bons "features" pour mettre en lumière les anomalies : les méthodes de détection d'anomalies appliquées sur les signaux bruts ou leur transformée en ondelettes, n'ont certes pas été totalement optimisées mais  ne donnent pas de bons résultats dasn ce cas précis. Par contre, la transformée de Fourier rapide met bien en exergue les anomalies sur le cas de ces données. On ne peut pas en tirer de généralités : sur les données simulées de télémesures du calepin disponible  [ici](http://localhost:8888/notebooks/High-Dimensional-Learning/DonneesSimulees/HDSTAT-Python-Anomaly-Detection.ipynb), la transformation dans une base d'ondelettes est pertinente pour la détection d'anomalies dans les données fonctionnelles. Il est donc important de bien connaitre les données et le type d'anomaies que l'on souhaite détecter. 

